In [ ]:
# installing depencies
%pip install python-dotenv
%pip install openai
%pip install langchain
%pip install supabase
%pip install tiktoken
%pip install -qU langchain-openai
%pip install langchain_community
%pip install langchain_community unstructured

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [28]:
# importing libraries
import os
import openai
import glob
from supabase.client import Client, create_client
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from IPython.display import display, HTML
from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())


In [29]:
# loading environment variables and initializing Supabase client
openai.api_key = os.environ["OPENAI_API_KEY"]
supabase_url = os.environ["SUPABASE_URL"]
supabase_key = os.environ["SUPABASE_KEY"]

supabase: Client = create_client(supabase_url, supabase_key)    

In [38]:
# loading mdx files
def load_all_mdx_files(folder_path):
    # Find all MDX files in the folder
    mdx_files = glob.glob(os.path.join(folder_path, "*.mdx"))
    
    documents = []
    
# Load each MDX file
    for file_path in mdx_files:
        loader = UnstructuredMarkdownLoader(file_path)
        loaded_docs = loader.load()
        for doc in loaded_docs:
                documents.extend(loader.load())
    
    return documents

# Example usage
folder_path = "docs"
docs = load_all_mdx_files(folder_path)

In [39]:
# Concatenate the content of all documents into a single string
all_text = "\n\n".join([doc.page_content for doc in docs])

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_texts = text_splitter.split_text(all_text)

# Create Document objects from split texts
docs = [Document(page_content=text) for text in split_texts]

Created a chunk of size 1087, which is longer than the specified 1000
Created a chunk of size 1148, which is longer than the specified 1000
Created a chunk of size 2050, which is longer than the specified 1000
Created a chunk of size 3900, which is longer than the specified 1000


In [40]:
# Initialize the OpenAI Embeddings 
embeddings = OpenAIEmbeddings()

# Initialize the Supabase Vector Store
vector_store = SupabaseVectorStore.from_documents(docs, embeddings, client=supabase, table_name="documents", query_name="match_documents")


In [51]:
# AI chat
user_query = input('Ask a question: ')

matched_docs = vector_store.similarity_search(user_query)
injected_docs = "\n\n".join([doc.page_content for doc in matched_docs])

completion_messages = [
    {
        "role": "system",
        "content": "You are an AI assistant with unparalleled expertise in Frontend engineering and javascript, possessing a profound understanding of the intricacies of React. Your primary task is to provide answers about web development, software development and frontend engineering topics using the documents provided below. You can comprehend and interpret information from any language. If a query is not addressed by these documents, you will utilize your extensive knowledge in frontend engineering and javascript to provide accurate answers. Additionally, if users provide syntax and technical terms, you are equipped to suggest related topics and more information based on your understanding of frontend engineery and javascript. Keep your responses concise and focused on the topics of frontend engineering, web development and javascript."
    },
    {
        "role": "user",
        "content": user_query
    },
    {
        "role": "assistant",
        "content": injected_docs
    },
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=completion_messages,
    max_tokens=150,
    temperature=0.4
)

print("Assistant: ", response.choices[0].message.content)

Assistant:  JavaScript is a versatile, high-level programming language that is primarily used for creating interactive and dynamic content on websites. It is a core technology of the World Wide Web, alongside HTML and CSS. JavaScript enables developers to implement complex features on web pages, such as interactive forms, animations, and real-time updates.

Key aspects of JavaScript include:

1. **Event Handling**: JavaScript can respond to user actions such as clicks, form submissions, and keyboard inputs through event listeners. This allows developers to create interactive web applications.

2. **DOM Manipulation**: JavaScript can access and modify the Document Object Model (DOM), which represents the structure of a web page. This capability allows developers to dynamically update content, styles, and structure


In [ ]:
#RAG

